# Find optimal portfolio weights

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import math
import datetime 
import statsmodels.formula.api as smf 
from datetime import date, time, datetime, timedelta
import seaborn as sns
import random

In [ ]:
qb = QuantBook()

tickers = ['MS','XOM']
symbols = []

for ticker in tickers:
    symbols.append(qb.AddEquity(ticker, Resolution.Daily).Symbol)

start_time = datetime(2017,1,1)
end_time = datetime(2018,1,1)
history = qb.History(symbols, start_time, end_time, Resolution.Daily)
df = history['close'].unstack(level=0)
df.columns = tickers

In [ ]:
# compute daily returns and form covariance matrix of the returns
dg = df.pct_change(1).dropna()

# covariance matrix of the returns
dg_cov= dg.cov()

# This function calculate the portfolio return and standard deviation for a given weight
def port(wt):
    cov_matrix = dg_cov
    return_matrix = pd.DataFrame (data = dg.mean())
    return_matrix.index = cov_matrix.index
    wt.index = cov_matrix.index
    v = cov_matrix.dot(wt)
    u = return_matrix.T.dot(wt)
    port_var = wt.T.dot(v).values
    port_return = u.values
    
    return [port_return, port_var]

In [ ]:
# simulate portfolio weights.
ret = [] # portfolio returns
sig = [] # portfolio return volatility
weights = []
n = 200 #number of simulation run
for i in range(n):
    w1 = random.randint(0,n)
    for j in range (n):
        w2 = n-w1
        wt = pd.DataFrame([w1/n, w2/n])
        weights.append(wt)
        
        r = port(wt)[0]
        v = port(wt)[1]
        #print (r, v) 
        
        ret = np.append(ret, r)
        sig = np.append(sig, math.sqrt(v))

# Generate plot with 200 samples

plt.figure(figsize=[17,9])
plt.scatter(sig, ret, color ="green")
plt.xlabel("Volatility", fontsize='large', fontweight='bold')
plt.ylabel("Expected Return",fontsize='large', fontweight='bold')
plt.title("Mean Variance Portfolio Consisting of " + str(tickers))

plt.show()

In [ ]:
# find the optimal portfolio weights
sortedsig = [i[0] for i in sorted(enumerate(sig), key=lambda x: abs(0.008-x[1]))]
idx = sortedsig[0] # array index of the closest point to 0.008
print(weights[idx])

In [ ]:
# compute volatility
weights1 = pd.DataFrame([0, 1])
weights2 = pd.DataFrame([1, 0])
sigma1 = math.sqrt(port(weights1)[1])
sigma2 = math.sqrt(port(weights2)[1])
print(sigma1, sigma2)

In [ ]:
tickers = ['MS','XOM']
symbols = []

for ticker in tickers:
    symbols.append(qb.AddEquity(ticker, Resolution.Daily).Symbol)

start_time = datetime(2017,1,1)
end_time = datetime(2018,1,1)
history = qb.History(symbols, start_time, end_time, Resolution.Daily)
df = history['close'].unstack(level=0)
df.columns = tickers